# Problem 1 Hypopnea event details

In [1]:
import xml.etree.ElementTree as ET
import pyedflib
import numpy as np
import csv
import pandas as pd

In [2]:
tree = ET.parse("mesa.xml")  #passing file file
root = tree.getroot()
Headers=['Duration','Input','Name','Start']
Data=[]

In [3]:
#finding required data
for m in root.iter('ScoredEvent'): 
    name=m.find('Name').text
    if name == 'Hypopnea': 
        for member in name:
                duration = m.find('Duration').text
                Data.append(duration)

                Input = m.find('Input').text
                Data.append(Input)

                name = m.find('Name').text
                Data.append(name)

                start = m.find('Start').text
                Data.append(start)

In [4]:
#saving the data into dataframe
df1= pd.DataFrame(Data)
DF= pd.DataFrame(df1.values.reshape(-1, 4),columns=Headers)
DF.drop_duplicates(keep='first', inplace=True)
DF.to_csv('myDataFrame.csv', index=False)
DF.head()

,Duration,Input,Name,Start
0,12.3,Flow,Hypopnea,6114.7
8,11.1,Flow,Hypopnea,6188.9
16,12,Flow,Hypopnea,5489.7
24,10.7,Flow,Hypopnea,5552.1
32,11.2,Flow,Hypopnea,5573.4


# problem 2 reading edf file and extracting data

In [29]:
#ref: https://github.com/holgern/pyedflib very helpfull

f = pyedflib.EdfReader("edf mesa-sleep-0001.edf")
n= f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n,f.getNSamples()[0]))
signal_labels_row = ",".join(signal_labels)

In [30]:
#reading the signal and saving it 
print (signal_labels_row)
flow_index = 8
buf = f.readSignal(flow_index)
buf

EKG,EOG-L,EOG-R,EMG,EEG1,EEG2,EEG3,Pres,Flow,Snore,Thor,Abdo,Leg,Therm,Pos,EKG_Off,EOG-L_Off,EOG-R_Off,EMG_Off,EEG1_Off,EEG2_Off,EEG3_Off,Pleth,OxStatus,SpO2,HR,DHR


array([-0.0698558 , -0.08926528, -0.09018082, ...,  0.00375372,
        0.00155642,  0.00027466])

In [31]:
f._close()# very important. pyedflib doesn't clean up in between files if the same handle is used
n = f.getNSamples()[flow_index]
Values_epoch = []
for i in np.arange(n):
    i = i/32
    Values_epoch.append(i)

In [32]:
print(len(buf))
print(len(Values_epoch))#pushing extracted data into pd.dataframe
Done = pd.DataFrame(
    {'Epoch': Values_epoch,
     'Flow': buf,
    })

1382368
1382368


In [33]:
Done.to_csv('mesa-sleep-0001-profusion.xml', index=False)
Done.head()

,Epoch,Flow
0,0.00000,-0.069856
1,0.03125,-0.089265
2,0.06250,-0.090181
3,0.09375,-0.091646
4,0.12500,-0.093843


# Problem 3 getting Stages event detail

In [36]:
#passing file
tree_1 = ET.parse('mesa-sleep-0001-nsrr.xml')
root_1 = tree_1.getroot()

In [37]:
#containes for storing extrated data
duration_3q=[]
stages_3q=[]
start_3q=[]
type_3q=[]

In [39]:
#extracting required data. ref: http://docs.activestate.com/activepython/3.1/python/library/xml.etree.elementtree.html

for event1 in root_1.findall('./ScoredEvents/ScoredEvent/[EventType="Stages|Stages"]'):
    duration_1 = event1.find('Duration').text
    duration_3q.append(duration_1)
    
    stages_1 = event1.find('EventType').text
    stages_3q.append(stages_1)
    
    start_1 = event1.find('Start').text
    start_3q.append(start_1)
    
    type_1 = event1.find('EventConcept').text
    type_3q.append(type_1)

In [43]:
#pushing the data into dataframe
D = pd.DataFrame(
    {'Duration': duration_3q,
     'Stages': type_3q,
     'Start': start_3q,
     'Types': stages_3q,
    })

In [44]:
D.to_csv('mesa-sleep.xml.csv', index=False)

In [45]:
D.head()

,Duration,Stages,Start,Types
0,5190.0,Wake|0,0.0,Stages|Stages
1,30.0,Stage 1 sleep|1,5190.0,Stages|Stages
2,90.0,Wake|0,5220.0,Stages|Stages
3,60.0,Stage 1 sleep|1,5310.0,Stages|Stages
4,90.0,Stage 2 sleep|2,5370.0,Stages|Stages
